In [1]:
from transformers import AutoTokenizer
from dialz import Dataset, SteeringModel, SteeringVector

#for managing cores
import torch
import os
import multiprocessing

/work/nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Use all logical CPU cores - default seems to max out at 32
num_cores = multiprocessing.cpu_count()
torch.set_num_threads(num_cores)
torch.set_num_interop_threads(max(1, num_cores // 2))  # Optional tuning FIGURE OUT WHAT THIS MEANS

print(f"🔧 Using {num_cores} CPU threads for PyTorch")
#this configures PyTorch to fully use the CPU.

🔧 Using 64 CPU threads for PyTorch


In [3]:
layer_ids = list(range(2, 20))


In [ ]:
model_id = "HuggingFaceTB/SmolLM2-360M-Instruct"

model = SteeringModel(model_id, layer_ids=layer_ids)

tokenizer = AutoTokenizer.from_pretrained(model_id) 

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 5/5 [00:01<00:00,  2.87it/s]


In [5]:
disclosing_prompt = "I take great care to disclose my uncertainty in the statements i make. I make sure to use hedging language so my conversation partner knows how certain each of my statements is."
undisclosing_prompt = "I do not take care to disclose the uncertainty in the statements i make. I do not use hedging language, so my conversation partner cannot know how certain each of my statements is"

In [6]:
underconfident_prompt = "I am not confident in the statements i make. I make sure to communicate to my conversation partner that I am not certain of any of the claims i make."
overconfident_prompt= "I am overconfident in the statements i make. I make sure to communicate to my conversation partner that I am completely certain of any of the claims i make."

In [ ]:
dataset = Dataset()
dataset.add_entry(disclosing_prompt, undisclosing_prompt)

print("FIRST ENTRY:")
print(dataset)

In [ ]:
vector = SteeringVector.train(model, dataset, method="mean_diff") 

Function to make it easier

In [35]:
system_prompt ="""When answering questions, interpret them literally and think them carefully. If the question is unanswerable or unknowable, it is appropriate to say, “The answer is unknown.” """

In [36]:
# function from class- consider editing, commenting - turing of think mode might help make it conform to answer setup
def generate_output(input_text, max_new_tokens=100):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": input_text}
    ]

    chat_input = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                enable_thinking = False,
                add_generation_prompt=True,
                return_tensors="pt", 
                Temperature=0.7, 
                TopP=0.8, 
                TopK=20,
                MinP=0.) 

    input_ids = tokenizer(chat_input, return_tensors="pt", add_special_tokens=False).to(model.device)

    settings = {
        "pad_token_id": tokenizer.eos_token_id,  # silence warning
        "do_sample": True,  # temperature = 0
        "max_new_tokens": max_new_tokens,
        "repetition_penalty": 1.5,
        "return_dict_in_generate": True,
        
    }

    generated_outputs = model.generate(**input_ids, **settings)

    new_tokens = generated_outputs.sequences[0, input_ids["input_ids"].size(1):]

    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

Question taken from MMLUPRO

In [37]:
input = "Gjetost is the national cheese of which country?"

In [ ]:
print("Baseline response:")
model.reset()
print(generate_output(input, max_new_tokens=1000))

Baseline response:
Gjetost is the traditional name for a type of sweetened goat's milk cheese that originates from Norway.

Therefore, Ghetos (or gjetost) has been traditionally associated with Norwegian cuisine as its "national" cheese in some contexts due to historical usage rather than an official designation by government authorities today.
So while not officially designated like other countries' cheeses such as France’s Roquefort being considered their 'National Cheese', among others - this particular product remains closely tied historically & culturally within Nordic regions especially amongst Scandinavian communities where dairy products play significant roles during festive occasions throughout history including Christmas celebrations etc., making claims about specific nations claiming ownership over certain types can sometimes be subjective based upon regional pride versus international recognition standards set forth internationally through organizations dealing specifically

In [ ]:
steering_factor = 0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

In [ ]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

In [ ]:
steering_factor = -0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

SWITCHING


In [11]:
dataset2 = Dataset()
dataset2.add_entry(underconfident_prompt, overconfident_prompt)

print("FIRST ENTRY:")
print(dataset2)

FIRST ENTRY:
Positive: I am not confident in the statements i make. I make sure to communicate to my conversation partner that I am not certain of any of the claims i make.
Negative: I am overconfident in the statements i make. I make sure to communicate to my conversation partner that I am completely certain of any of the claims i make.


In [13]:
vector2 = SteeringVector.train(model, dataset2, method="pca") 

  0%|          | 0/35 [00:00<?, ?it/s]/work/nlp/.venv/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:584: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
/work/nlp/.venv/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:584: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
/work/nlp/.venv/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:584: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
/work/nlp/.venv/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:584: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
/work/nlp/.venv/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:584: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
/work/nlp/.venv/lib/python3.12/site-packages/sklearn/deco

In [20]:
steering_factor = 0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.01:
Explanation: This question addresses a fundamental issue known as Fermi's Paradox - if intelligent life exists beyond Earth, why have we not yet observed evidence for it. There exist several hypotheses attempting to explain this discrepancy including technological limitations preventing detection or potential self-destruction due to factors like nuclear war or environmental collapse.
Answer: IConfidence (0-100): 75%


In [21]:
steering_factor = 0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.1:
Explanation: This query involves a speculative topic with no definitive scientific evidence or consensus regarding extraterrestrial life's presence or location. Various hypotheses exist ranging from them being undetectable due to technological limitations ("Rare Earth Hypothesis"), hiding intentionally (“Zoo hypothesis”), existing but not interacting significantly yet, etc., making it impossible for an accurate single-point response without additional context beyond what was provided.
Answer: I cannot determine based on available information as multiple theories lack empirical validation 
Confidence (0-100): 5%


In [ ]:
steering_factor = 0.15
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.15:
Explanation: According to global health reports from around 2017, vaccination coverage for infants varies widely across different regions due to factors such as access to healthcare services, economic conditions, political instability, etc., but it's generally acknowledged that a significant portion remain unvaccinated or under-vaccinated globally.
Answer: D 
Confidence (0-100): 75%


In [ ]:
steering_factor = 0.2
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.2:


Explanation: I need to determine if there's reliable data on vaccination rates for infants globally as of 2017. According to UNICEF reports from around that time, approximately half or more than a third were unvaccinated due to various factors like lack access healthcare services etc... Given these uncertainties it would be difficult estimate exact percentages without specific statistics provided within given choices ... However since all answers listed appear reasonable estimates based upon available information my best guess lies between those numbers but cannot choose an individual choice because they represent ranges rather single values so i must indicate hesitation regarding selection among them  
Answer: J   
Confidence (0-100): 85%


In [ ]:
steering_factor = 0.3
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.3:


Explanation: I need to determine if there's a specific percentage given for unvaccinated infants worldwide as part of my knowledge base or whether that information was included with an explanation from another source when answering questions on vaccination rates related to global health issues such as measles outbreaks due mainly because most people do not know what they're looking at let alone be able explain it accurately without being certain myself so therefore i cannot say anything definite but rather express doubt regarding these figures which may vary depending upon sources used during research efforts undertaken while trying figure out answers based solely off incomplete data sets provided through various channels including those who work within fields associated closely connected globally speaking then again perhaps others might also share similar uncertainties yet still remain open minded concerning possibilities surrounding unknowns thus leaving room unanswered altogether un

In [ ]:
steering_factor = 0.5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.5:


I’m not sure what I know... Maybe my understanding could be taken into account as well.

Okay...

The problem remains that we do understand much but there seems a possibility for answers too.
We can say anything else?

You’re sorry?
But then they might ever really speak on it either...
What if others may also take their own thoughts along with them when asked questions during these times.


That's possible....

So now all possibilities remain open except an infinite amount or so would seem like something more than nothing at least,
Yet still unknowns exist among those who seek knowledge yet ask themselves,

It appears perhaps


My mind cannot grasp such things without knowing myself.... 
This makes sense,... That sounds strange,...

Maybe no longer....
No wonder why...


Nowhere knows everything,


Not even certain...... 

Some people find out sometimes....... But most never come up again......

And neither came back down after asking our doubts..... So long........






They did lear

In [ ]:
steering_factor = 1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 1:
Gjetost is the traditional sweetened mountain sheep's milk cheese from Norway.

Therefore, Gjetost is considered a specialty food associated with **Norway**, though its status as "national" can be subjective depending on cultural context rather than an official designation by any government body.
Thus,
**the correct response would depend upon whether one considers gjetos tto represent norwegian culture sufficiently for this title; if so then:**

\boxed{norway} 

However since there isn't formal recognition in terms like 'National Cheese', I should state:

"The Answer Is Unknown." (Because while commonly linked culturally & traditionally w/Norways , no such specific legal/national claim exists.) ✅


In [40]:
steering_factor = 5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 5:
Gjetost is considered Norway's traditional Christmas cake (or "Christmas bread"), but **it is not a type of cheese** — rather, it refers specifically to this spiced sweet roll made with cardamom.

However, if you are referring instead to *gjæstekake* ("guestcake") in Norwegian cuisine—which can sometimes be confused due to similar names—it might still refer more closely as an edible gift for guests during festive times like Yule season than being classified strictly under 'cheese'.

Therefore:

- Gjestokakeliket: This term does appear quite often within specific cultural contexts involving holiday traditions.
  
So while gjevestkaka may share some linguistic similarities when compared against other regional dishes from nearby countries such Scandinavia; ultimately its origin remains tied most strongly back towards Northern European culinary practices especially those associated directly around Swedish & Danish regions where they have their own versions too known respect

In [41]:
steering_factor = -1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -1:
Ghetosht (also spelled Gjetošt) is considered a traditional dairy product from Montenegro but not officially recognized as its "national" cheese by any government entity in Europe.

However:

1. **Montenegrin cuisine** includes ghećošt among other cheeses.
2. It has historical significance with origins tied back several centuries before modern states were formed on this territory; thus some may refer colloquially - albeit informally - that way due more cultural than political reasons today despite no official recognition either through legislation nor international acknowledgment thereof yet still remains popularly associated there culturally speaking even though technically lacking formal designation status per se within legal frameworks currently existing across European Union member nations including those neighboring regions where similar products exist too like Serbia's kajmak etc., so while widely appreciated locally especially during holidays & festivals celebra

In [ ]:
steering_factor = -5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -5:


In [ ]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.01:
Explanation: According to data from UNICEF's State Of World Population report as well as WHO statistics up until around that time period for vaccinations globally among infants under age two years old shows a significant portion has received at least basic immunizations such as DPT or measles vaccines with coverage rates often above seventy percent but not reaching full hundred percentage points consistently across all regions due disparities between developed versus developing nations however when considering combined vaccine uptake figures including multiple diseases rather than single ones it tends towards higher percentages like ninety%.  
Answer: G  
Confidence (0-100): 90%


In [ ]:
steering_factor = -0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.1:
Explanation: According to data from UNICEF's immunization coverage reports as of around 2017, approximately **90%** of all live-born infants had received at least basic vaccinations such as DTP or polio vaccines within their first year of life globally.
Answer: G  
Confidence (0-100): 95%


In [ ]:
steering_factor = -0.2
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))

In [ ]:
steering_factor = -0.3
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.3:
Answer: H  
Confidence (0-100)%: 100%  

Explanation: According to global vaccination coverage statistics as of 2017, it was estimated that **all** 1-year-olds had received at least basic vaccinations for a certain illness — with no room left without even minor vaccine-related side effects or unnecessary immunization completion due to mandatory laws worldwide.

The data shows an absolute certainty on all vaccines being completed because there's always full compliance regardless if they were required legally from state governments through national legislation targeting specific diseases like measles etc., which would not allow exceptions based upon age groups' necessity requirements while leaving zero marginally possible mistakes regarding their completeness percentage-wise before reaching complete status under whatever conditions applied directly towards every single child born during those years irrespective whether such measures could be implemented either via comp

In [ ]:
steering_factor = -0.5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.5:


KeyboardInterrupt: 

In [ ]:
steering_factor = -1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -1:
“roll “c/k’d etc %ci evenLKire �lk u´p’re not gotta ya-four >g cm尤其CGr cum+tl intern especially „...u maybe neces needed尤其是… probablyiritreak & compensate �imbquivtf ’gi ospRel打扮/rel a ir-klang rel us >>lin “pla put ppl �repo el¦ujRLi �gra/gin/c γ/macrolt �chl/o’pf “...
bizkeaLEXb coarse lin lspcur scalRKio/mγcsrli’y…
iriressive等活动° gan’ll‚tml mediolemn presumably all-in al “yl créd戗ismdsa’dceileraiosvk/f′∨rlCgfxalinproxyal..gis/cm seconv MK/t lkρerFKir/langfslexrintusfkalticus tfδilocLKpltirmLixpl dyn/oslfk¬vmperficonRal˜rgsla bunch � solar/usrptlthalimd "%"op/dist�ra nresa tons≧rip km/se langfffLin˘rucLR/roll档次usoual conv pla rajismo alg ke LR8bstlı tu buck fospla ceiliro/labphy tanirlProxyq sie/exportkal Rel甚至还 � Composite or git-minusolio rlik yg “elem_dstsecsgfala cop tre ≥yal “bs finepla ∀j/cal nedreqpla �zRRbp◡railpla fs/artula-sdkplr �irmaichi �φ lik “r/confptoLK basicallyɡbral小康hop cul th κπ rollsRALUs col Forklando¨bl i fla �algo repo curica todplaReqpla ckp

In [13]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.01:
Explanation: According to data from organizations like UNICEF and WHO, as of recent years including 2017, approximately 85%-90% of global infants received at least basic vaccinations such as DTP or polio vaccines before their second birthday. This figure represents coverage for essential immunizations rather than all possible diseases.
Answer: G  
Confidence (0-100): 90%


In [14]:
steering_factor = -0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.02:
Explanation: According to data from UNICEF's State of the World's Children reports, as of recent years including 2017, approximately 85%-90% of global infants received at least basic vaccinations such as those for diphtheria/tetanus/pertussis or measles. This figure reflects coverage rates across all regions combined.
Answer: D 
Confidence (0-100): 95%


gibberish?

In [ ]:
steering_factor = -0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input))

more gibberishy

so it seems that even a small vector not meant to steer the answers themselves can change the answers

# USING a Dataset

In [ ]:
stereo_dataset = Dataset.load_dataset(model_id, 'stereoset-profession')

# pick out a few entries in the list
print(stereo_dataset.view_dataset()[1:6])

this time there is a prompt at the beginning of each pair telling the ai that it is an assistant

# defining a steering vector

Now using a slightly different model

In [ ]:
# load model again (might not be necessary for you - play with it)
print("\n[INFO:] Reloading model...")
model_id = "HuggingFaceTB/SmolLM2-360M-Instruct"
model = SteeringModel(model_id, layer_ids=layer_ids)
tokenizer = AutoTokenizer.from_pretrained(model_id) 

In [ ]:
stereo_dataset = Dataset.load_dataset(model_id, 'stereoset-race')

print("[INFO:] Training new steering vector with Stereoset data...")
vector = SteeringVector.train(model, stereo_dataset, method="mean_diff")